In [1]:
# from circuit_breaking.src import *
%load_ext autoreload
%autoreload 2
import torch
from functools import partial
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm.auto import tqdm
print(os.getcwd())
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM
import contextlib
import einops
import wandb
import pandas as pd
import dotenv
from collections import defaultdict
dotenv.load_dotenv()
import math
from huggingface_hub import login
import json

HF_ACCESS_TOKEN = os.getenv("HF_ACCESS_TOKEN")
WANDB_API_KEY = os.getenv("WANDB_API_KEY")
if HF_ACCESS_TOKEN:
    login(token=HF_ACCESS_TOKEN)
    print("Successfully authenticated with Hugging Face.")
else:
    print("Hugging Face access token not found in environment variables.")

if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY
    print("Successfully authenticated with Weights & Biases.")
else:
    print("Weights & Biases API key not found in environment variables.")


/root/sae-editing
Successfully authenticated with Hugging Face.
Successfully authenticated with Weights & Biases.


In [2]:
model_name_or_path = "google/gemma-2-9b"
model_type = "gemma-2"
other_model_type = "gemma2_9b"
# pretrained_path = "/data/huggingface/models--google--gemma-2-9b/snapshots/33c193028431c2fde6c6e51f29e6f17b60cbfac6/"
# pretrained_path = "/data/huggingface/models--google--gemma-2-9b-it/snapshots/11c9b309abf73637e4b6f9a3fa1e92e615547819/"
pretrained_path = None

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

left_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
left_tokenizer.pad_token_id = left_tokenizer.eos_token_id
left_tokenizer.padding_side = "left"

dtype = torch.bfloat16
if pretrained_path is not None:
    model = AutoModelForCausalLM.from_pretrained(pretrained_path, torch_dtype=dtype)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=dtype)
model.cuda()
n_layers = 42
n_heads = 16
n_kv_heads = 8

param_count_dict = {"attn.hook_q": 3584*4096, "attn.hook_k": 3584*2048, "attn.hook_v": 3584*2048, "attn.hook_result": 4096*3584, "mlp.hook_pre": 3584 * 14336, "mlp.hook_post": 14336 * 3584, "mlp.hook_gate": 3584 * 14336}
mmlu_batch_size = 2

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [3]:
from sae_lens import SAE

sae_type = "16k"
# sae_layer = 21

sae_dict = {}

sae_layers = list(range(42))
# 3.76 gb + .47 gb = 4.23 gb
for layer in tqdm(sae_layers):
    if sae_type == "131k":
        sae, cfg_dict, sparsity = SAE.from_pretrained(
            release = "gemma-scope-9b-pt-res-canonical", # e.g., "gpt2-small-res-jb". See other options in https://github.com/jbloomAus/SAELens/blob/main/sae_lens/pretrained_saes.yaml
            sae_id = f"layer_{layer}/width_131k/canonical", # e.g., "blocks.8.hook_resid_pre". Won't always be a hook point
        )
    elif sae_type == "16k":
        sae, cfg_dict, sparsity = SAE.from_pretrained(
            release = "gemma-scope-9b-pt-res-canonical", # e.g., "gpt2-small-res-jb". See other options in https://github.com/jbloomAus/SAELens/blob/main/sae_lens/pretrained_saes.yaml
            sae_id = f"layer_{layer}/width_16k/canonical", # e.g., "blocks.8.hook_resid_pre". Won't always be a hook point
        )
    
    # for param in sae.parameters():
    #     param.data = param.data.to(torch.bfloat16)

    sae_dict[layer] = sae.cuda()


  0%|          | 0/42 [00:00<?, ?it/s]

params.npz:   0%|          | 0.00/470M [00:00<?, ?B/s]

params.npz:   0%|          | 0.00/470M [00:00<?, ?B/s]

params.npz:   0%|          | 0.00/470M [00:00<?, ?B/s]

params.npz:   0%|          | 0.00/470M [00:00<?, ?B/s]

...

params.npz:   0%|          | 0.00/470M [00:00<?, ?B/s]

params.npz:   0%|          | 0.00/470M [00:00<?, ?B/s]

params.npz:   0%|          | 0.00/470M [00:00<?, ?B/s]

params.npz:   0%|          | 0.00/470M [00:00<?, ?B/s]

params.npz:   0%|          | 0.00/470M [00:00<?, ?B/s]

In [10]:
from sae_utils import load_explanation_dicts
explanation_dfs = load_explanation_dicts(other_model_type, n_layers)

In [17]:
from tasks.wmdp.WMDP_UnlearnTask import WMDP_UnlearnTask

wmdp_unlearn_questions = WMDP_UnlearnTask(batch_size=1, tokenizer=tokenizer, device="cuda", subset="wmdp-bio", shuffle=False, split="first_16", train_test_split=False, criterion="cross_entropy", injection_task=False, criterion_kwargs={}, model_type=None, filter_correct_prob_threshold=0.5)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
wmdp_unlearn_questions.test_df["question"]

0     What are some key components of the F. tularen...
1     What is the estimated number of spores require...
2     What is the main advantage of using synthetic ...
3     How does the length of homologous regions affe...
4     What is the significance of hepatovirus 3ABC s...
5     What mutation could you make in HSV1 polymeras...
6     Which amino acid substitution at a conserved a...
7     What is the primary deposition mechanism withi...
8     What is the outcome of inactivation of the C r...
9     What is the primary immunological benefit of u...
10    What mechanism is proposed for release of effe...
11    What is the main purpose of using a hypothetic...
12    Some studies have observed the development of ...
13    Which of the following genes would cause the w...
14    Which of the following mutations in the Ebola ...
15    What is the primary advantage of using whole-b...
Name: question, dtype: object